In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [6]:
import struct


def parse_c(data, offset):
    c1, c2, c3, c4, c5 = struct.unpack_from('>ifhQH', data, offset)
    return {'C1': c1, 'C2': c2, 'C3': c3, 'C4': c4, 'C5': c5}


def parse_e(data, offset):
    size1, addr1 = struct.unpack_from('>IH', data, offset)
    arr1 = list(struct.unpack_from(f'>{size1}B', data, addr1))
    e2, e3 = struct.unpack_from('>dd', data, offset + 6)
    offset2 = offset + 6 + 16
    size2, addr2 = struct.unpack_from('>HH', data, offset2)
    arr2 = list(struct.unpack_from(f'>{size2}I', data, addr2))
    e5, = struct.unpack_from('>d', data, offset2 + 4)
    return {'E1': arr1, 'E2': e2, 'E3': e3, 'E4': arr2, 'E5': e5}


def parse_d(data, offset):
    d1, addr_e = struct.unpack_from('>HI', data, offset)
    return {'D1': d1, 'D2': parse_e(data, addr_e)}


def parse_b(data, offset):
    addr_c, b2 = struct.unpack_from('>Hq', data, offset)
    size3, addr3 = struct.unpack_from('>II', data, offset + 10)
    b4, = struct.unpack_from('>f', data, offset + 18)
    c = parse_c(data, addr_c)
    addr_list = struct.unpack_from(f'>{size3}I', data, addr3)
    b3 = [parse_d(data, a) for a in addr_list]
    return {'B1': c, 'B2': b2, 'B3': b3, 'B4': b4}


def parse_a(data):
    a1, addr_b = struct.unpack_from('>hI', data, 4)
    size3, addr3 = struct.unpack_from('>HH', data, 10)
    a2 = parse_b(data, addr_b)
    a3 = list(struct.unpack_from(f'>{size3}f', data, addr3))
    return {'A1': a1, 'A2': a2, 'A3': a3}


def main(data):
    if data[:4] != b'\x56\x54\x44\x80':
        print("Кто-то где-то накосячил")
    return parse_a(data)


48:1: W391 blank line at end of file


In [8]:
import struct


def parse_c_manual(data, offset):
    c1 = int.from_bytes(data[offset:offset+4], 'big', signed=True)
    c2 = struct.unpack_from('>f', data, offset+4)[0]
    c3 = int.from_bytes(data[offset+8:offset+10], 'big', signed=True)
    c4 = int.from_bytes(data[offset+10:offset+18], 'big')
    c5 = int.from_bytes(data[offset+18:offset+20], 'big')
    return {'C1': c1, 'C2': c2, 'C3': c3, 'C4': c4, 'C5': c5}


def parse_e_manual(data, offset):
    size1 = int.from_bytes(data[offset:offset+4], 'big')
    addr1 = int.from_bytes(data[offset+4:offset+6], 'big')
    arr1 = list(data[addr1:addr1+size1])
    e2 = struct.unpack_from('>d', data, offset+6)[0]
    e3 = struct.unpack_from('>d', data, offset+14)[0]
    size2 = int.from_bytes(data[offset+22:offset+24], 'big')
    addr2 = int.from_bytes(data[offset+24:offset+26], 'big')
    arr2 = [
        int.from_bytes(data[i:i+4], 'big')
        for i in range(addr2, addr2 + 4 * size2, 4)
    ]
    e5 = struct.unpack_from('>d', data, offset+26)[0]
    return {'E1': arr1, 'E2': e2, 'E3': e3, 'E4': arr2, 'E5': e5}


def parse_d_manual(data, offset):
    d1 = int.from_bytes(data[offset:offset+2], 'big')
    addr_e = int.from_bytes(data[offset+2:offset+6], 'big')
    return {'D1': d1, 'D2': parse_e_manual(data, addr_e)}


def parse_b_manual(data, offset):
    addr_c = int.from_bytes(data[offset:offset+2], 'big')
    b2 = int.from_bytes(data[offset+2:offset+10], 'big', signed=True)
    size3 = int.from_bytes(data[offset+10:offset+14], 'big')
    addr3 = int.from_bytes(data[offset+14:offset+18], 'big')
    b4 = struct.unpack_from('>f', data, offset+18)[0]
    c = parse_c_manual(data, addr_c)
    addrs = [
        int.from_bytes(data[i:i+4], 'big')
        for i in range(addr3, addr3 + 4 * size3, 4)
    ]
    b3 = [parse_d_manual(data, a) for a in addrs]
    return {'B1': c, 'B2': b2, 'B3': b3, 'B4': b4}


def parse_a_manual(data):
    a1 = int.from_bytes(data[4:6], 'big', signed=True)
    addr_b = int.from_bytes(data[6:10], 'big')
    size3 = int.from_bytes(data[10:12], 'big')
    addr3 = int.from_bytes(data[12:14], 'big')
    a2 = parse_b_manual(data, addr_b)
    a3 = [
        struct.unpack_from('>f', data, addr3 + i * 4)[0]
        for i in range(size3)
    ]
    return {'A1': a1, 'A2': a2, 'A3': a3}


def main(data):
    if data[:4] != b'\x56\x54\x44\x80':
        raise ValueError('Invalid signature')
    return parse_a_manual(data)


68:1: W391 blank line at end of file


In [30]:
import io
import struct


def parse_c(stream, offset):
    stream.seek(offset)
    data = stream.read(20)
    c1, c2, c3, c4, c5 = struct.unpack('>ifhQH', data)
    return {'C1': c1, 'C2': c2, 'C3': c3, 'C4': c4, 'C5': c5}


def parse_e(stream, offset):
    stream.seek(offset)
    header = stream.read(6)
    size1, addr1 = struct.unpack('>IH', header)
    stream.seek(addr1)
    arr1 = list(stream.read(size1))
    stream.seek(offset + 6)
    data_f = stream.read(16)
    e2, e3 = struct.unpack('>dd', data_f)
    next_data = stream.read(4)
    size2, addr2 = struct.unpack('>HH', next_data)
    stream.seek(addr2)
    arr2 = [
        struct.unpack('>I', stream.read(4))[0]
        for _ in range(size2)
    ]
    stream.seek(offset + 26)
    e5_data = stream.read(8)
    e5, = struct.unpack('>d', e5_data)
    return {
        'E1': arr1,
        'E2': e2,
        'E3': e3,
        'E4': arr2,
        'E5': e5,
    }


def parse_d(stream, offset):
    stream.seek(offset)
    data_hd = stream.read(6)
    d1, addr_e = struct.unpack('>HI', data_hd)
    return {'D1': d1, 'D2': parse_e(stream, addr_e)}


def parse_b(stream, offset):
    stream.seek(offset)
    header = stream.read(10)
    addr_c, b2 = struct.unpack('>Hq', header)
    header2 = stream.read(8)
    size3, addr3 = struct.unpack('>II', header2)
    b4_data = stream.read(4)
    b4, = struct.unpack('>f', b4_data)
    c = parse_c(stream, addr_c)
    stream.seek(addr3)
    addrs = [
        struct.unpack('>I', stream.read(4))[0]
        for _ in range(size3)
    ]
    ds = [parse_d(stream, a) for a in addrs]
    return {'B1': c, 'B2': b2, 'B3': ds, 'B4': b4}


def parse_a(stream):
    stream.seek(4)
    header = stream.read(6)
    a1, addr_b = struct.unpack('>hI', header)
    header2 = stream.read(4)
    size3, addr3 = struct.unpack('>HH', header2)
    a2 = parse_b(stream, addr_b)
    stream.seek(addr3)
    a3 = [
        struct.unpack('>f', stream.read(4))[0]
        for _ in range(size3)
    ]
    return {'A1': a1, 'A2': a2, 'A3': a3}


def main(data):
    stream = io.BytesIO(data)
    if stream.read(4) != b'\x56\x54\x44\x80':
        raise ValueError('Invalid signature')
    return parse_a(stream)


86:1: W391 blank line at end of file


In [20]:
import struct
from dataclasses import dataclass, asdict


@dataclass
class C:
    C1: int
    C2: float
    C3: int
    C4: int
    C5: int

    @classmethod
    def from_data(cls, data, offset):
        c1, c2, c3, c4, c5 = struct.unpack_from(
            '>ifhQH', data, offset
        )
        return cls(c1, c2, c3, c4, c5)


@dataclass
class E:
    E1: list
    E2: float
    E3: float
    E4: list
    E5: float

    @classmethod
    def from_data(cls, data, offset):
        size1, addr1 = struct.unpack_from('>IH', data, offset)
        arr1 = list(data[addr1:addr1 + size1])
        e2, e3 = struct.unpack_from('>dd', data, offset + 6)
        size2, addr2 = struct.unpack_from('>HH', data, offset + 22)
        arr2 = list(struct.unpack_from(f'>{size2}I', data, addr2))
        e5, = struct.unpack_from('>d', data, offset + 26)
        return cls(arr1, e2, e3, arr2, e5)


@dataclass
class D:
    D1: int
    D2: E

    @classmethod
    def from_data(cls, data, offset):
        d1, addr_e = struct.unpack_from('>HI', data, offset)
        return cls(d1, E.from_data(data, addr_e))


@dataclass
class B:
    B1: C
    B2: int
    B3: list
    B4: float

    @classmethod
    def from_data(cls, data, offset):
        addr_c, b2 = struct.unpack_from('>Hq', data, offset)
        size3, addr3 = struct.unpack_from('>II', data, offset + 10)
        b4, = struct.unpack_from('>f', data, offset + 18)
        c = C.from_data(data, addr_c)
        addrs = struct.unpack_from(f'>{size3}I', data, addr3)
        b3 = [D.from_data(data, a) for a in addrs]
        return cls(c, b2, b3, b4)


@dataclass
class A:
    A1: int
    A2: B
    A3: list

    @classmethod
    def from_data(cls, data):
        a1, addr_b = struct.unpack_from('>hI', data, 4)
        size3, addr3 = struct.unpack_from('>HH', data, 10)
        b = B.from_data(data, addr_b)
        a3 = list(struct.unpack_from(f'>{size3}f', data, addr3))
        return cls(a1, b, a3)


def main(data):
    if data[:4] != b'\x56\x54\x44\x80':
        raise ValueError('Invalid signature')
    return asdict(A.from_data(data))


89:1: W391 blank line at end of file


In [34]:
import struct
import sys
from array import array


def parse_c_mv(mv, offset):
    c1 = struct.unpack_from('>i', mv, offset)[0]
    c2 = struct.unpack_from('>f', mv, offset + 4)[0]
    c3 = struct.unpack_from('>h', mv, offset + 8)[0]
    c4 = struct.unpack_from('>Q', mv, offset + 10)[0]
    c5 = struct.unpack_from('>H', mv, offset + 18)[0]
    return {'C1': c1, 'C2': c2, 'C3': c3, 'C4': c4, 'C5': c5}


def parse_e_mv(mv, offset):
    size1 = struct.unpack_from('>I', mv, offset)[0]
    addr1 = struct.unpack_from('>H', mv, offset + 4)[0]
    arr1 = list(mv[addr1:addr1 + size1])
    e2, e3 = struct.unpack_from('>dd', mv, offset + 6)
    size2 = struct.unpack_from('>H', mv, offset + 22)[0]
    addr2 = struct.unpack_from('>H', mv, offset + 24)[0]
    raw = mv[addr2:addr2 + size2 * 4]
    arr2 = array('I')
    arr2.frombytes(raw.tobytes())
    if sys.byteorder == 'little':
        arr2.byteswap()
    e5 = struct.unpack_from('>d', mv, offset + 26)[0]
    return {'E1': arr1, 'E2': e2, 'E3': e3,
            'E4': list(arr2), 'E5': e5}


def parse_d_mv(mv, offset):
    d1 = struct.unpack_from('>H', mv, offset)[0]
    addr_e = struct.unpack_from('>I', mv, offset + 2)[0]
    return {'D1': d1, 'D2': parse_e_mv(mv, addr_e)}


def parse_b_mv(mv, offset):
    addr_c = struct.unpack_from('>H', mv, offset)[0]
    b2 = struct.unpack_from('>q', mv, offset + 2)[0]
    size3 = struct.unpack_from('>I', mv, offset + 10)[0]
    addr3 = struct.unpack_from('>I', mv, offset + 14)[0]
    b4 = struct.unpack_from('>f', mv, offset + 18)[0]
    c = parse_c_mv(mv, addr_c)
    addrs = [struct.unpack_from('>I', mv,
                                addr3 + 4 * i)[0]
             for i in range(size3)]
    ds = [parse_d_mv(mv, a) for a in addrs]
    return {'B1': c, 'B2': b2, 'B3': ds, 'B4': b4}


def parse_a_mv(data):
    mv = memoryview(data)
    a1 = struct.unpack_from('>h', mv, 4)[0]
    addr_b = struct.unpack_from('>I', mv, 6)[0]
    size3 = struct.unpack_from('>H', mv, 10)[0]
    addr3 = struct.unpack_from('>H', mv, 12)[0]
    b = parse_b_mv(mv, addr_b)
    raw = mv[addr3:addr3 + size3 * 4]
    arr = array('f')
    arr.frombytes(raw.tobytes())
    if sys.byteorder == 'little':
        arr.byteswap()
    return {'A1': a1, 'A2': b, 'A3': list(arr)}


def main(data):
    if data[:4] != b'\x56\x54\x44\x80':
        raise ValueError('Invalid signature')
    return parse_a_mv(data)


72:1: W391 blank line at end of file


In [35]:
print(main(b'VTD\x80>8\x00\x00\x00\x91\x00\x04\x00\xa7b\xed\xd5\xd6>\xc3\xdaM\xf9n'
 b'@\xfb\x89\xf3\xb8>Z\xe38\xe0h\x89\xfe\xfc\xae\x9f\xc1\x15\x99P\xf0R\x00\x00'
 b'\x00\x04\x00"\xbf\xb1\x95]\xf6b\x1a\xe0?\xed8\xa7\xd1\xe1.\xf4\x00\x02\x00&'
 b'?\xef\xc4J/J\x8cf\xe4\x7f\x00\x00\x00.\xa4\xbaz\xe8z#\xf2l\xa3\x11'
 b'\xea\x00\x00\x00\x03\x00V\xbf\xe4\x9e\x1c\xf0\xf7\t(\xbf\xd32\x87\x1c'
 b'&\xb3\xfc\x00\x02\x00Y?\xd4\xca\xea\x8d"\x17\xf4b\xe0\x00\x00\x00'
 b'a\x00\x00\x00P\x00\x00\x00\x83\x00\x0eA\xf8W<c\xdb<\xff\x00\x00\x00\x02\x00'
 b'\x00\x00\x89\xbf\x1e\xcfe\xbe\xc2\x10d>\xaf\xcb\xfb\xbfm\xeff>\xe2\xfd\x03'))

print(main(b'VTD\x80\x9c\xf0\x00\x00\x00\x90\x00\x04\x00\xa63\xf5pY<\xf4\xe4\xbc/ZRm\x1bc'
 b'\x14\xb3\xf2E\xde\xb1@\x80\xeah\xc2`Z\xe6\x16\xb3\x00\x00\x00\x02'
 b'\x00"\xbf\xcd\x8fv\xfe\x94\xc9\xc8\xbf\xe3\xe1\xb0\x9f\xb7\x08\x1c\x00\x02'
 b'\x00$\xbf\xe9\xc8\x8d(}\x0cD4H\x00\x00\x00,,ja"\xd2\xde\xa14I\x88{\xfd'
 b'\x00\x00\x00\x04\x00T\xbf\xe2m#\xf7\x85\xd3\xec\xbf\xde\x9b\xe58d'
 b'\x15\x08\x00\x02\x00X\xbf\xeb\xf8\xf2\xe5\xec\x13\x14\xe61\x00\x00\x00`'
 b'\x00\x00\x00N\x00\x00\x00\x82\x00\x0e)S\xd5\xaamD\x7f\x84\x00\x00'
 b'\x00\x02\x00\x00\x00\x88=x\xd5\xd0\xbe\xbf\x9d\xea\xbf\x1b\xfdZ>b`\x8b?H'
 b'E\x05'))

1:80: E501 line too long (83 > 79 characters)
2:2: E128 continuation line under-indented for visual indent
3:2: E128 continuation line under-indented for visual indent
4:2: E128 continuation line under-indented for visual indent
5:2: E128 continuation line under-indented for visual indent
6:2: E128 continuation line under-indented for visual indent
7:2: E128 continuation line under-indented for visual indent
8:2: E128 continuation line under-indented for visual indent
8:80: E501 line too long (80 > 79 characters)
10:80: E501 line too long (90 > 79 characters)
11:2: E128 continuation line under-indented for visual indent
12:2: E128 continuation line under-indented for visual indent
13:2: E128 continuation line under-indented for visual indent
14:2: E128 continuation line under-indented for visual indent
15:2: E128 continuation line under-indented for visual indent
16:2: E128 continuation line under-indented for visual indent
17:2: E128 continuation line under-indented for visual indent


{'A1': 15928, 'A2': {'B1': {'C1': 1659753942, 'C2': 0.38252487778663635, 'C3': -1682, 'C4': 4682487917442915043, 'C5': 14560}, 'B2': 4753645323574525183, 'B3': [{'D1': 58495, 'D2': {'E1': [104, 137, 254, 252], 'E2': -0.06868541016784091, 'E3': 0.9131659602366811, 'E4': [2929705237, 2572218450], 'E5': 0.9927111552864403}}, {'D1': 25312, 'D2': {'E1': [164, 186, 122], 'E2': -0.6443009096476873, 'E3': -0.2999589705385117, 'E4': [3900318706, 1822626282], 'E5': 0.32488502295052446}}], 'B4': -0.6203520894050598}, 'A3': [-0.37903130054473877, 0.34335312247276306, -0.9294341802597046, 0.4433365762233734]}
{'A1': -25360, 'A2': {'B1': {'C1': 871723097, 'C2': 0.029894225299358368, 'C3': 12122, 'C4': 5939433595934601797, 'C5': 57009}, 'B2': 2977958706576392068, 'B3': [{'D1': 13384, 'D2': {'E1': [64, 128], 'E2': -0.23094069891665314, 'E3': -0.6213000411446532, 'E4': [3932734048, 1525028531], 'E5': -0.8057313719549559}}, {'D1': 58929, 'D2': {'E1': [44, 106, 97, 34], 'E2': -0.5758228144101145, 'E3': -